# Occultations in Reflected Light

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import starry

In [ ]:
def occultation_is_simple(xo_, yo_, ro, b, tol1=1e-8, tol2=1e-8, tol3=1e-5):
    """Is the occultation simple (solvable in terms of classic starry integrals only)?"""
    # Rotate into Convention II
    xo = -yo_
    yo = xo_

    # Shorthand
    b2 = b ** 2
    b4 = b2 ** 2
    x2 = xo ** 2
    y2 = yo ** 2
    r2 = ro ** 2

    # Check if it's a full occultation
    if np.sqrt(x2 + y2) < (ro - 1):
        return True

    # Check if we are very close to full / new phase
    if np.abs(b) > 1 - tol1:
        return True

    # Check if we are very close to half phase
    if np.abs(b) < tol2:
        term = r2 - x2

        if term <= 0:
            # No roots
            return True

        term = np.sqrt(term)
        ya = yo + term
        yb = yo - term

        # Check for roots on terminator
        for y in (ya, yb):
            if (y > -1) and (y < 1):
                return False

        # There is no intersection
        return True

    # Occultor along the axis? (special case)
    if np.abs(yo) < tol3:

        # Solve the quadratic
        term = x2 + (1 - b2) * (1 - r2)

        if term <= 0:
            # No roots
            return True

        # Find the two roots
        term = np.sqrt(term)
        fac = b / (b2 - 1)
        xa = fac * (b * xo + term)
        xb = fac * (b * xo - term)

        # Check for roots on the terminator
        for x in (xa, xb):
            if b < 0:
                if (x > 0) and (x < -b):
                    return False
            else:
                if (x < 0) and (x > -b):
                    return False

        # There is no intersection
        return True

    # Compute the coefficients of the quartic determining
    # the intersection of the occultor and the terminator
    A = (1 - b2) ** 2 / b4
    B = 4 * xo * (1 / b2 - 1)
    C = 2 * (-1 + r2 - x2 + y2 + b2 * (1 - r2 + 3 * x2 + y2)) / b2
    D = -4 * xo * (1 - r2 + x2 + y2)
    E = (1 - r2 + x2) ** 2 - 2 * (1 + r2 - x2) * y2 + y2 ** 2
    p = np.array([A, B, C, D, E])

    # Count the roots
    if b > 0:
        nroots = starry._c_ops.nroots(p, -1, 0)
    else:
        nroots = starry._c_ops.nroots(p, 0, 1)

    # If there are no roots, we're good!
    if nroots == 0:
        return True
    else:
        return False

In [ ]:
def intersection(xo, yo, ro, b, tol=1e-6):
    """Return the points of intersection."""

    # Compute the coefficients of the quartic
    b2 = b ** 2
    b4 = b2 ** 2
    x2 = xo ** 2
    y2 = yo ** 2
    r2 = ro ** 2
    A = (1 - b2) ** 2
    B = -4 * xo * (1 - b2)
    C = -2 * (b4 + r2 - 3 * x2 - y2 - b2 * (1 + r2 - x2 + y2))
    D = -4 * xo * (b2 - r2 + x2 + y2)
    E = b4 - 2 * b2 * (r2 - x2 + y2) + (r2 - x2 - y2) ** 2
    p = np.array([A, B, C, D, E])

    # Compute the real roots numerically
    roots = [r.real for r in np.roots(p) if np.abs(r.imag) < tol]
    res = []

    for x in roots:
        y = b * np.sqrt(1 - x ** 2)
        if np.abs((y - yo) ** 2 - ro ** 2 + (x - xo) ** 2) < tol:
            res.append((x, y))
    return res

In [ ]:
def plot(ax, xo, yo, ro, b):
    """Plot the occultation geometry."""

    # Body
    ax.add_artist(plt.Circle((0, 0), radius=1, fc="none", ec="k"))

    # Terminator
    x = np.linspace(-1, 1, 1000)
    y = b * np.sqrt(1 - x ** 2)
    ax.plot(x, y, color="k", lw=1)
    ax.plot(x, -y, color="k", alpha=0.1, lw=1)
    ax.fill_between(x, -np.sqrt(1 - x ** 2), y, color="k", alpha=0.5)

    # Simple occultation?
    if occultation_is_simple(xo, yo, ro, b):
        color = "g"
    else:
        color = "r"

    # Occultor
    ax.add_artist(plt.Circle((xo, yo), radius=ro, fc="none", ec=color))

    # Roots
    for x, y in intersection(xo, yo, ro, b):
        ax.plot(x, y, "ro", ms=3, markeredgecolor="k")

        if color == "g":
            raise ValueError("Disagreement on number of roots!")

    return ax

In [ ]:
xo = 0
yo = 1.31
ro = 0.25
b = 0.5
fig, ax = plt.subplots(3, 8, figsize=(12, 4))
for axis in ax.flatten():
    plot(axis, xo, yo, ro, b)
    axis.axis("off")
    axis.set_aspect(1)
    axis.set_ylim(-1.25, 1.25)
    axis.set_xlim(-1.25, 1.25)
    yo -= 0.11

In [ ]:
xo = 0.5
yo = 1.31
ro = 0.25
b = 0.5
fig, ax = plt.subplots(3, 8, figsize=(12, 4))
for axis in ax.flatten():
    plot(axis, xo, yo, ro, b)
    axis.axis("off")
    axis.set_aspect(1)
    axis.set_ylim(-1.25, 1.25)
    axis.set_xlim(-1.25, 1.25)
    yo -= 0.11

In [ ]:
xo = 0.95
yo = 1.31
ro = 0.25
b = 0.5
fig, ax = plt.subplots(3, 8, figsize=(12, 4))
for axis in ax.flatten():
    plot(axis, xo, yo, ro, b)
    axis.axis("off")
    axis.set_aspect(1)
    axis.set_ylim(-1.25, 1.25)
    axis.set_xlim(-1.25, 1.25)
    yo -= 0.11

In [ ]:
xo = 0
yo = 1.31
ro = 0.25
b = -0.5
fig, ax = plt.subplots(3, 8, figsize=(12, 4))
for axis in ax.flatten():
    plot(axis, xo, yo, ro, b)
    axis.axis("off")
    axis.set_aspect(1)
    axis.set_ylim(-1.25, 1.25)
    axis.set_xlim(-1.25, 1.25)
    yo -= 0.11

In [ ]:
xo = 0.5
yo = 1.31
ro = 0.25
b = -0.5
fig, ax = plt.subplots(3, 8, figsize=(12, 4))
for axis in ax.flatten():
    plot(axis, xo, yo, ro, b)
    axis.axis("off")
    axis.set_aspect(1)
    axis.set_ylim(-1.25, 1.25)
    axis.set_xlim(-1.25, 1.25)
    yo -= 0.11

In [ ]:
xo = 0.95
yo = 1.31
ro = 0.25
b = -0.5
fig, ax = plt.subplots(3, 8, figsize=(12, 4))
for axis in ax.flatten():
    plot(axis, xo, yo, ro, b)
    axis.axis("off")
    axis.set_aspect(1)
    axis.set_ylim(-1.25, 1.25)
    axis.set_xlim(-1.25, 1.25)
    yo -= 0.11

In [ ]:
xo = 0
yo = 1.31
ro = 1.25
b = 0.5
fig, ax = plt.subplots(3, 8, figsize=(12, 4))
for axis in ax.flatten():
    plot(axis, xo, yo, ro, b)
    axis.axis("off")
    axis.set_aspect(1)
    axis.set_ylim(-1.5, 1.5)
    axis.set_xlim(-1.5, 1.5)
    yo -= 0.11

In [ ]:
xo = 1.0
yo = 1.31
ro = 1.25
b = 0.5
fig, ax = plt.subplots(3, 8, figsize=(12, 4))
for axis in ax.flatten():
    plot(axis, xo, yo, ro, b)
    axis.axis("off")
    axis.set_aspect(1)
    axis.set_ylim(-1.5, 1.5)
    axis.set_xlim(-1.5, 1.5)
    yo -= 0.11

In [ ]:
xo = 2.0
yo = 1.31
ro = 1.25
b = 0.5
fig, ax = plt.subplots(3, 8, figsize=(12, 4))
for axis in ax.flatten():
    plot(axis, xo, yo, ro, b)
    axis.axis("off")
    axis.set_aspect(1)
    axis.set_ylim(-1.5, 1.5)
    axis.set_xlim(-1.5, 1.5)
    yo -= 0.11

In [ ]:
xo = 0
yo = 1.31
ro = 1.25
b = -0.5
fig, ax = plt.subplots(3, 8, figsize=(12, 4))
for axis in ax.flatten():
    plot(axis, xo, yo, ro, b)
    axis.axis("off")
    axis.set_aspect(1)
    axis.set_ylim(-1.5, 1.5)
    axis.set_xlim(-1.5, 1.5)
    yo -= 0.11

In [ ]:
xo = 1.0
yo = 1.31
ro = 1.25
b = -0.5
fig, ax = plt.subplots(3, 8, figsize=(12, 4))
for axis in ax.flatten():
    plot(axis, xo, yo, ro, b)
    axis.axis("off")
    axis.set_aspect(1)
    axis.set_ylim(-1.5, 1.5)
    axis.set_xlim(-1.5, 1.5)
    yo -= 0.11

In [ ]:
xo = 2.0
yo = 1.31
ro = 1.25
b = -0.5
fig, ax = plt.subplots(3, 8, figsize=(12, 4))
for axis in ax.flatten():
    plot(axis, xo, yo, ro, b)
    axis.axis("off")
    axis.set_aspect(1)
    axis.set_ylim(-1.5, 1.5)
    axis.set_xlim(-1.5, 1.5)
    yo -= 0.11

In [ ]:
xo = 1e-1
yo = 1.31
ro = 1.25
b = 0.49
fig, ax = plt.subplots(3, 8, figsize=(12, 4))
for axis in ax.flatten():
    plot(axis, xo, yo, ro, b)
    axis.axis("off")
    axis.set_aspect(1)
    axis.set_ylim(-1.5, 1.5)
    axis.set_xlim(-1.5, 1.5)
    yo -= 0.11

In [ ]:
xo = 1e-1
yo = 51.1
ro = 50
b = 0.5
fig, ax = plt.subplots(3, 8, figsize=(12, 4))
for axis in ax.flatten():
    plot(axis, xo, yo, ro, b)
    axis.axis("off")
    axis.set_aspect(1)
    axis.set_ylim(-1.5, 1.5)
    axis.set_xlim(-1.5, 1.5)
    yo -= 0.09

In [ ]:
xo = 0
yo = 51.1
ro = 50
b = -0.5
fig, ax = plt.subplots(3, 8, figsize=(12, 4))
for axis in ax.flatten():
    plot(axis, xo, yo, ro, b)
    axis.axis("off")
    axis.set_aspect(1)
    axis.set_ylim(-1.5, 1.5)
    axis.set_xlim(-1.5, 1.5)
    yo -= 0.09

In [ ]:
xo = 0.25
yo = 1.1
ro = 0.25
b = -0.999999
fig, ax = plt.subplots(3, 8, figsize=(12, 4))
for axis in ax.flatten():
    plot(axis, xo, yo, ro, b)
    axis.axis("off")
    axis.set_aspect(1)
    axis.set_ylim(-1.5, 1.5)
    axis.set_xlim(-1.5, 1.5)
    yo -= 0.09

In [ ]:
xo = 0.25
yo = 1.1
ro = 0.25
b = 0.999999
fig, ax = plt.subplots(3, 8, figsize=(12, 4))
for axis in ax.flatten():
    plot(axis, xo, yo, ro, b)
    axis.axis("off")
    axis.set_aspect(1)
    axis.set_ylim(-1.5, 1.5)
    axis.set_xlim(-1.5, 1.5)
    yo -= 0.09

In [ ]:
xo = 0
yo = 1.1
ro = 0.25
b = 0.99999
fig, ax = plt.subplots(3, 8, figsize=(12, 4))
for axis in ax.flatten():
    plot(axis, xo, yo, ro, b)
    axis.axis("off")
    axis.set_aspect(1)
    axis.set_ylim(-1.5, 1.5)
    axis.set_xlim(-1.5, 1.5)
    yo -= 0.09

In [ ]:
xo = 0.1
yo = 1.1
ro = 0.25
b = 0
fig, ax = plt.subplots(3, 8, figsize=(12, 4))
for axis in ax.flatten():
    plot(axis, xo, yo, ro, b)
    axis.axis("off")
    axis.set_aspect(1)
    axis.set_ylim(-1.5, 1.5)
    axis.set_xlim(-1.5, 1.5)
    yo -= 0.09

In [ ]:
xo = 0
yo = 1.21
ro = 1.25
b = 0
fig, ax = plt.subplots(3, 8, figsize=(12, 4))
for axis in ax.flatten():
    plot(axis, xo, yo, ro, b)
    axis.axis("off")
    axis.set_aspect(1)
    axis.set_ylim(-1.5, 1.5)
    axis.set_xlim(-1.5, 1.5)
    yo -= 0.09

In [ ]:
# Pathological case!
xo = 0
yo = -0.75
ro = 1.3
b = 0.5
fig, ax = plt.subplots(1, figsize=(12, 6))
plot(ax, xo, yo, ro, b)
ax.axis("off")
ax.set_aspect(1)
ax.set_ylim(0, 1.01)
ax.set_xlim(-1.01, 1.01);